## Sugestão de cidade utilizando o Índice de Criminalidade do Estado do Rio Grande do Sul
### Para o trabalho final da disciplina de Sistemas de Recomendação foi escolhido os bancos de dados Índice de Criminalidade (RS, 2023) e Estimativas Populacionais (RS, 2022). Os dados são reais e estão disponíveis no portal Dados Abertos RS do Estado do Rio Grande do Sul.

Indíce de Criminalidade no RS: [https://dados.rs.gov.br/dataset/indicadores-criminais-de-2023](https://dados.rs.gov.br/dataset/indicadores-criminais-de-2023)

Estimativas Populacionais no RS: [https://dados.rs.gov.br/dataset/dee-4259/resource/ce259dd9-c479-4a18-90b3-40098e6deb26](https://dados.rs.gov.br/dataset/dee-4259/resource/ce259dd9-c479-4a18-90b3-40098e6deb26)

### Pré-processamento nos Datasets

Algumas transformações necessarias nos dados:
- Os os índices de Criminalidade por região foram fornecidos separados por mês e os nomes das colunas demasiadamente grandes.

- Os dados populacionais por ano foram fornecidos com preenchimento inválido no ano de 2012 e os nomes das colunas eram demasiadamente extensos.

- Criação de uma chave estrangeira para relacionar as tabelas.

In [936]:
import pandas as pd

#### Abertura dos arquivos de criminalidade por cidade em cada mês.

In [937]:
# Declarando os meses
meses = ['janeiro', 'fevereiro', 'marco', 'abril', 'maio', 'junho', 'julho', 'agosto', 'setembro', 'outubro', 'novembro', 'dezembro']

dfs = []

# Lendo os arquivos conforme os meses e adicionando a lista de dataframes
for mes in meses:
        arquivo = pd.read_csv(f'data\\tocsv\\{mes}.csv', delimiter=';')
        dfs.append(arquivo)

# Concatenando todos os dataframes ignorando o index
df = pd.concat(dfs, ignore_index=True)

# Agrupando por município e ibge e somando os valores
df_criminal = df.groupby(['municipios', 'ibge']).sum().reset_index()

# Salvando o dataframe em um arquivo csv
df.to_csv('data\\tocsv\\total.csv', sep=';', index=False)

df_criminal


,municipios,ibge,homicidio_doloso,total_vitimas_homicidio_doloso,latrocinio,furtos,abigeato,furto_veiculo,roubos,roubo_veiculo,estelionato,delitos_armas_municoes,entorpecente_posse,entorpecente_trafico,vitimas_latrocinio,vitimas_lesao_corporal_morte,total_vitimas_crimes_violentos
0,acegua,4300034,0,0,0,20,8,0,3,0,27,4,3,0,0,0,0
1,agua santa,4300059,1,1,0,20,3,4,1,0,13,2,0,0,0,0,2
2,agudo,4300109,1,1,0,102,2,2,3,1,51,3,7,10,0,0,1
3,ajuricaba,4300208,0,0,0,26,1,0,1,0,28,2,3,0,0,0,0
4,alecrim,4300307,1,1,0,36,10,0,2,0,17,10,0,4,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,vista alegre do prata,4323606,0,0,0,3,0,0,0,0,8,3,0,0,0,0,0
493,vista gaucha,4323705,1,1,0,11,0,0,0,0,6,2,0,0,0,0,1
494,vitoria das missoes,4323754,0,0,0,20,3,0,0,0,7,1,4,0,0,0,0
495,westfalia,4323770,0,0,0,10,2,1,0,0,20,1,1,0,0,0,0


#### Abertura do arquivo de quantidade de população por cidade em cada ano.

In [938]:
# Lendo o arquivo de população
df_pop = pd.read_csv('data\\tocsv\\populacao.csv', delimiter=';')

#df_pop.info() mostra que as colunas de 4 em diante são do tipo object e para fazer operações matemáticas é necessário converter para inteiro

# Substituindo os valores de '.' e '-' por 0 e convertendo para inteiro
for coluna in df_pop.columns[4:]:
    df_pop[coluna] = df_pop[coluna].str.replace('.', '').str.replace('-', '0').astype(int)

df_pop.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   municipios  497 non-null    object 
 1   ibge        497 non-null    int64  
 2   latitude    497 non-null    float64
 3   longitude   497 non-null    float64
 4   2010        497 non-null    int32  
 5   2011        497 non-null    int32  
 6   2012        497 non-null    int32  
 7   2013        497 non-null    int32  
 8   2014        497 non-null    int32  
 9   2015        497 non-null    int32  
 10  2016        497 non-null    int32  
 11  2017        497 non-null    int32  
 12  2018        497 non-null    int32  
 13  2019        497 non-null    int32  
 14  2020        497 non-null    int32  
 15  2021        497 non-null    int32  
dtypes: float64(2), int32(12), int64(1), object(1)
memory usage: 39.0+ KB


### União dos dois banco de dados

In [939]:
# O merge é feito com base na coluna ibge como forma de chave estrangeira com o método inner
df_merged = pd.merge(df_criminal, df_pop, how = 'inner', on = 'ibge').drop(columns=['municipios_y'])
df_merged.rename(columns={'municipios_x': 'municipios'}, inplace=True)

df_merged.info()

df_merged


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 31 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   municipios                      497 non-null    object 
 1   ibge                            497 non-null    int64  
 2   homicidio_doloso                497 non-null    int64  
 3   total_vitimas_homicidio_doloso  497 non-null    int64  
 4   latrocinio                      497 non-null    int64  
 5   furtos                          497 non-null    int64  
 6   abigeato                        497 non-null    int64  
 7   furto_veiculo                   497 non-null    int64  
 8   roubos                          497 non-null    int64  
 9   roubo_veiculo                   497 non-null    int64  
 10  estelionato                     497 non-null    int64  
 11  delitos_armas_municoes          497 non-null    int64  
 12  entorpecente_posse              497 

,municipios,ibge,homicidio_doloso,total_vitimas_homicidio_doloso,latrocinio,furtos,abigeato,furto_veiculo,roubos,roubo_veiculo,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,acegua,4300034,0,0,0,20,8,0,3,0,...,4539,4520,4564,4483,4472,4412,4487,4516,4540,4505
1,agua santa,4300059,1,1,0,20,3,4,1,0,...,3858,3898,3959,3922,3977,4013,4057,4107,4093,4256
2,agudo,4300109,1,1,0,102,2,2,3,1,...,16731,16838,16851,16701,16595,16475,16537,16556,16760,16612
3,ajuricaba,4300208,0,0,0,26,1,0,1,0,...,7389,7431,7299,7241,7279,7325,7546,7485,7584,7447
4,alecrim,4300307,1,1,0,36,10,0,2,0,...,7074,6891,6814,6598,6594,6569,6513,6435,6301,6403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,vista alegre do prata,4323606,0,0,0,3,0,0,0,0,...,1578,1539,1582,1630,1677,1645,1648,1704,1721,1746
493,vista gaucha,4323705,1,1,0,11,0,0,0,0,...,2842,2828,2787,2790,2802,2834,2885,2940,2987,3002
494,vitoria das missoes,4323754,0,0,0,20,3,0,0,0,...,3453,3403,3415,3448,3383,3439,3389,3438,3397,3405
495,westfalia,4323770,0,0,0,10,2,1,0,0,...,2864,2957,2974,3007,3039,3088,3136,3125,3226,3257


### Cálculo da taxa de criminalidade a cada 1000 mil habitantes.
 Para que os dados das grandes cidades, com numerosa população, não tenham impacto nas análises estatísticas, calculasse a taxa de criminalidade percapita. Assim a comparação entre Índice de Criminalidade e População mantém a proporcionalidade.

In [940]:
# Lista das colunas que precisam ser calculadas
colunas_taxa = df_criminal.columns[2:]

# Calculando a taxa de criminalidade para cada mês
for coluna_taxa in colunas_taxa:
    for coluna_pop in df_pop.columns[4:]:
            
            # Calculando a taxa de criminalidade por 100.000 habitantes
            df_merged[f'taxa_{coluna_taxa}'] = df_criminal[coluna_taxa] / df_pop[coluna_pop] * 100000

# Salvando o dataframe em um arquivo csv
df_merged.to_csv('data\\tocsv\\dados_criminalidade_população.csv', sep=';', index=False)


## Redução de Dimensionalidade
### O dataset agora tem dados suficientes para ranquear as cidades em mais seguras e não seguras com dados proporcionais a população. Entretanto, existem muitas colunas. A alta dimensionalidade deixará o processo de aprendizado confuso criando um sobreajuste nos dados. Então é aplicado o PCA, uma técnida de redução de dimensionalidade.


O PCA será aplicado somente nas colunas referentes as taxas de criminalidade, pois os números absolutos são dados brutos e somente serviram para o cálculo das taxas. Com isto, os componentes principais irão explicar o comportamento da criminalidade nas cidades com a maior variabilidade das taxas.

In [941]:
# Lista das colunas extras que não são necessárias
colunas_extras = df_merged.columns[2:-15]

# Removendo as colunas extras
df_taxa = df_merged.drop(colunas_extras, axis=1)

# Salvando o dataframe em um arquivo csv
df_taxa.to_csv('data\\tocsv\\dados_taxa_criminalidade.csv', sep=';', index=False)

df_taxa



,municipios,ibge,taxa_homicidio_doloso,taxa_total_vitimas_homicidio_doloso,taxa_latrocinio,taxa_furtos,taxa_abigeato,taxa_furto_veiculo,taxa_roubos,taxa_roubo_veiculo,taxa_estelionato,taxa_delitos_armas_municoes,taxa_entorpecente_posse,taxa_entorpecente_trafico,taxa_vitimas_latrocinio,taxa_vitimas_lesao_corporal_morte,taxa_total_vitimas_crimes_violentos
0,acegua,4300034,0.000000,0.000000,0.0,443.951165,177.580466,0.000000,66.592675,0.000000,599.334073,88.790233,66.592675,0.000000,0.0,0.0,0.000000
1,agua santa,4300059,23.496241,23.496241,0.0,469.924812,70.488722,93.984962,23.496241,0.000000,305.451128,46.992481,0.000000,0.000000,0.0,0.0,46.992481
2,agudo,4300109,6.019745,6.019745,0.0,614.013966,12.039490,12.039490,18.059234,6.019745,307.006983,18.059234,42.138213,60.197448,0.0,0.0,6.019745
3,ajuricaba,4300208,0.000000,0.000000,0.0,349.133879,13.428226,0.000000,13.428226,0.000000,375.990332,26.856452,40.284678,0.000000,0.0,0.0,0.000000
4,alecrim,4300307,15.617679,15.617679,0.0,562.236452,156.176792,0.000000,31.235358,0.000000,265.500547,156.176792,0.000000,62.470717,0.0,0.0,15.617679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,vista alegre do prata,4323606,0.000000,0.000000,0.0,171.821306,0.000000,0.000000,0.000000,0.000000,458.190149,171.821306,0.000000,0.000000,0.0,0.0,0.000000
493,vista gaucha,4323705,33.311126,33.311126,0.0,366.422385,0.000000,0.000000,0.000000,0.000000,199.866755,66.622252,0.000000,0.000000,0.0,0.0,33.311126
494,vitoria das missoes,4323754,0.000000,0.000000,0.0,587.371512,88.105727,0.000000,0.000000,0.000000,205.580029,29.368576,117.474302,0.000000,0.0,0.0,0.000000
495,westfalia,4323770,0.000000,0.000000,0.0,307.031010,61.406202,30.703101,0.000000,0.000000,614.062020,30.703101,30.703101,0.000000,0.0,0.0,0.000000


Para fins de demonstração, será usado somente uma amostra de 20 cidades.

In [942]:
# Selecionando 10 linhas aleatórias
df_sample= df_taxa.sample(n=10, axis=0, random_state=42)

In [943]:
import numpy as np

from statsmodels.datasets import get_rdataset
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [944]:
# Salvando os nomes dos municípios
municipios = df_sample['municipios']

# Selecionando as colunas do tipo float e que serão utilizadas para a análise do PCA
df_sample.drop(['municipios', 'ibge'], axis='columns', inplace=True)

df_sample.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 483 to 9
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   taxa_homicidio_doloso                10 non-null     float64
 1   taxa_total_vitimas_homicidio_doloso  10 non-null     float64
 2   taxa_latrocinio                      10 non-null     float64
 3   taxa_furtos                          10 non-null     float64
 4   taxa_abigeato                        10 non-null     float64
 5   taxa_furto_veiculo                   10 non-null     float64
 6   taxa_roubos                          10 non-null     float64
 7   taxa_roubo_veiculo                   10 non-null     float64
 8   taxa_estelionato                     10 non-null     float64
 9   taxa_delitos_armas_municoes          10 non-null     float64
 10  taxa_entorpecente_posse              10 non-null     float64
 11  taxa_entorpecente_trafico            1

In [945]:
# Normalizando os dados média 0 e desvio padrão 1
scaler = StandardScaler(with_std=True, with_mean=True)
data_scaled = pd.DataFrame(scaler.fit_transform(df_sample))
#data_scaled.describe()

In [946]:
# Aplicando a PCA
pca = PCA()
components = pca.fit_transform(data_scaled)

components.shape[1]

10

In [947]:
# Analise de como os componentes estão explicando os dados com a variância explicada
pca.explained_variance_ratio_

array([4.24796836e-01, 1.90845487e-01, 1.40782723e-01, 9.74176175e-02,
       7.63305412e-02, 4.78980940e-02, 1.47236931e-02, 4.41806007e-03,
       2.78694847e-03, 9.85344380e-34])

In [948]:
import plotly.express as px

In [949]:
# Plotando o gráfico da variância explicada acumulada
px.area(
    x=range(1, pca.explained_variance_ratio_.cumsum().shape[0] + 1),
    y=pca.explained_variance_ratio_.cumsum(),
    labels={"x": "# Components", "y": "Explained Variance"}
)

In [950]:
# Aplicando a PCA com 2 componentes
pca = PCA(n_components=2)
components = pca.fit_transform(data_scaled)

components.shape[1]

2

In [951]:
# Calculando as cargas dos componentes
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)

# Plotando o gráfico de dispersão com os componentes principais
fig = px.scatter(components, x=0, y=1, color=municipios, labels={'0': 'PC1', '1': 'PC2', 'color': 'Municipios'})

features = df_sample.columns

# Adicionando os vetores de carga ao gráfico
for i, feature in enumerate(features):
    fig.add_annotation(
        ax=0, ay=0,
        axref="x", ayref="y",
        x=loadings[i, 0],
        y=loadings[i, 1],
        showarrow=True,
        arrowsize=2,
        arrowhead=2,
        xanchor="right",
        yanchor="top"
    )
    
    fig.add_annotation(
        x=loadings[i, 0],
        y=loadings[i, 1],
        ax=0, ay=0,
        xanchor="center",
        yanchor="bottom",
        text=feature,
        yshift=5,
    )
# Invertendo o eixo y para que o gráfico fique na posição correta    
fig.update_yaxes(autorange="reversed")
fig.show()

In [952]:
import matplotlib.pyplot as plt
%matplotlib inline

In [953]:
""" i, j = 0, 1 # which components
scale_arrow = s_ = 2
components[:,1] *= -1
pca.components_[1] *= -1 # flip the y-axis
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
ax.scatter(components[:,0], components[:,1])
ax.set_xlabel('PC%d' % (i+1))
ax.set_ylabel('PC%d' % (j+1))
for k in range(pca.components_.shape[1]):
    ax.arrow(0, 0, s_*pca.components_[i,k], s_*pca.components_[j,k])
    ax.text(s_*pca.components_[i,k],
            s_*pca.components_[j,k],
            df_sample.columns[k]) """

" i, j = 0, 1 # which components\nscale_arrow = s_ = 2\ncomponents[:,1] *= -1\npca.components_[1] *= -1 # flip the y-axis\nfig, ax = plt.subplots(1, 1, figsize=(8, 8))\nax.scatter(components[:,0], components[:,1])\nax.set_xlabel('PC%d' % (i+1))\nax.set_ylabel('PC%d' % (j+1))\nfor k in range(pca.components_.shape[1]):\n    ax.arrow(0, 0, s_*pca.components_[i,k], s_*pca.components_[j,k])\n    ax.text(s_*pca.components_[i,k],\n            s_*pca.components_[j,k],\n            df_sample.columns[k]) "